In [ ]:
using LinearAlgebra, BenchmarkTools, LaTeXStrings, Plots, Plots.PlotMeasures, SparseArrays
using NAJ

In [ ]:
function butterfly3d(v, σ::Real, ρ::Real, β::Real)
    return [σ*(v[2]-v[1]), v[1]*(ρ - v[3])-v[2], v[1]*v[2]-β*v[3]]
end

In [ ]:
#tn, xn = ode_rk2((t, x)->butterfly3d(x, 10, 28, 8/3), 0.0, [0.0, 1.0, 1.0], 50000, 1.0e-3)
#plot(xn[1,:], xn[2,:], xn[3,:], label="RK2")

solver = InitialValueOdeProblem((t, x) -> butterfly3d(x, 10, 28, 8/3), 0.0, [0.0, 1.0, 1.0], 50000, 1.0e-3)
tn, xn = solve(solver, :rk2)
plot(xn[1,:], xn[2,:], xn[3,:], label="RK2")


In [ ]:
# tn, xn = ode_rk4((t, x)->butterfly3d(x, 10, 28, 8/3), 0.0, [0.0, 1.0, 1.0], 50000, 1.0e-3)
# plot(xn[1,:], xn[2,:], xn[3,:],dpi=300, label = "RK4")

tn, xn = solve(solver, :rk4)
plot(xn[1,:], xn[2,:], xn[3,:],dpi=300, label = "RK4")


In [ ]:
function eq_sho1d(y::Vector{<:Real}, k::Real, x0=0.0)
    return [y[2], -k*(y[1]-x0)]
end

sho1d = InitialValueOdeProblem((t, y)->eq_sho1d(y, 3, 1.0), 2.0, [1.1, 0.0], 10000, 1.0e-2,)
tn1, xn1 = solve(sho1d, :rk2)
tn2, xn2 = solve(sho1d, :rk4)
p1=plot(tn1, xn1[1,: ], label = :none, xlabel = L"t", ylabel = L"\textbf{Amplitude}", title = L"RK2", size=(400, 350))

p2=plot(tn2, xn2[1,: ], label = :none, xlabel = L"t", ylabel = L"\textbf{Amplitude}", title = L"RK4", size=(400, 350))
plot(p1, p2, size=(1000, 400), left_margin = [10mm 10mm], bottom_margin = 15px)

In [ ]:
function eq_dho1d(y::Vector{<:Real}, m=1.0, k=1.0, γ=1.0)
    return [y[2], -γ/m *y[2] - k/m*y[1]]
end

m, k, γ = 1.0, 1.0, 0.03
dho1d = InitialValueOdeProblem((t, y)->eq_dho1d(y, m, k, γ), 0.0, [0.0, 1.0], 10000, 1.0e-2)
tn1, xn1 = solve(dho1d, :euler)
tn2, xn2 = solve(dho1d, :rk2)
tn3, xn3 = solve(dho1d, :rk4)

p1=plot(tn1, xn1[1,: ], label = "Euler", xlabel = L"t", ylabel = L"\textbf{Amplitude}", title = L"\textbf{Underdamping}", lc=:red)#, size=(400, 350))
plot!(tn2, xn2[1,: ], label = "RK2", lc=:green)

plot!(tn3, xn3[1,: ], label = "RK4", lc=:blue)#, xlabel = L"t", ylabel = L"\textbf{Amplitude}", title = L"RK4", size=(400, 350))


λ = γ/(2*m)
ω = sqrt(k/m-γ^2/4/m)
rk2 = exp.(-λ .* tn3) .* sin.(ω .* tn3)
plot!(tn3, rk2, lc=:cyan, label = "analytic")
plot!(tn3, rk2-xn3[1, :], lc=:black, label = "analytic-rk4")
println("λ= $λ, ω = $ω")

m, k, γ = 1, 0.02, 0.6
dho1d2 = InitialValueOdeProblem((t, y)->eq_dho1d(y, m, k, γ), 0.0, [0.0, 1.0], 10000, 1.0e-2)
tn1, xn1 = solve(dho1d2, :euler)
tn2, xn2 = solve(dho1d2, :rk2)
tn3, xn3 = solve(dho1d2, :rk4)

p2=plot(tn1, xn1[1,: ], label = "Euler", xlabel = L"t", ylabel = L"\textbf{Amplitude}", title = L"\textbf{Overdamping}", lc=:red)#, size=(400, 350))
plot!(tn2, xn2[1,: ], label = "RK2", lc=:green)
plot!(tn3, xn3[1,: ], label = "RK4", lc=:blue)#


λ = γ/(2*m)
ω = sqrt(k/m+0.0im-γ^2/4/m)
println("λ= $λ, ω = $ω")
rk2 = 1.88982*exp.(-λ .* tn3) .* (exp.(imag(ω) .* tn3) - exp.(-imag(ω) .* tn3)) 
plot!(tn3, rk2, lc=:cyan, label = "analytic")
plot(p1, p2, dpi=300, left_margin = [10mm 10mm], bottom_margin = 15px, size=(1000, 450))

#savefig("dho1d_rk.png")

In [ ]:
function de_pendulum(y::Vector{<:Real}, g=9.8, l=2)
    return [y[2], -g/l*sin(y[1])]
end
g, l = 9.8, 5.0

pendulum = InitialValueOdeProblem((t, y)->de_pendulum(y, g, l), 0.0, [0.0, 0.04], 20000, 1.0e-2)
tn1, xn1 = solve(pendulum, :euler)
tn2, xn2 = solve(pendulum, :rk2)
tn3, xn3 = solve(pendulum, :rk4)

p1=plot(tn1, (180.0/π).* xn1[1,: ], label = "Euler", xlabel = L"t", ylabel = L"\textbf{Degree}", title = L"\textbf{Pendulum}", lc=:red, dpi=300)#, size=(400, 350))
plot!(tn2, (180.0/π).* xn2[1,: ], label = "RK2", lc=:green)

plot!(tn3, (180.0/π).* xn3[1,: ], label = "RK4", lc=:blue)#, xlabel = L"t", ylabel = L"\textbf{Amplitude}", title = L"RK4", size=(400, 350))
#savefig("ode_pendulum.png")


In [ ]:
function de_twobody(m1::Real, m2::Real, G::Real, xx::Vector{<:Real})
    x, y, v, w = xx[1:3], xx[4:6], xx[7:9], xx[10:12]
    r = y .- x
    rn = norm(r)^3
    k2 = (G*m2/rn) 
    k1 = (G*m1/rn) 
    A = zeros(12, 12)
    A[1:3, 7:9] = [1 0 0; 0 1 0; 0 0 1]
    A[4:6, 10:12] = [1 0 0; 0 1 0; 0 0 1]
    A[7:9, 1:3] = [-k2 0 0; 0 -k2 0; 0  0 -k2]
    A[7:9, 4:6] = [k2 0 0; 0 k2 0; 0 0 k2]
    A[10:12, 4:6] = [-k1 0 0; 0 -k1 0; 0 0 -k1]
    A[10:12, 1:3] = [k1 0 0; 0 k1 0; 0 0 k1]

    return A*xx
end

In [ ]:
me = 5.972e24
ms= 2.0e30
G =  6.67384e−11
v1 = 29.78e3
r0 =147.10e9
A=de_twobody(ms, me, G,[0, 0, 0, r0, 0, 0, 0, 0, 0, 0, v1, 0])
twobody = InitialValueOdeProblem((t, y)->de_twobody(ms, me, G, y), 0.0, [0, 0, 0, r0, 0, 0, 0, 0, 0, 0, v1, 0], 8760, 3600)
tn3, xn3 = solve(twobody, :rk4)

In [ ]:
p1 = plot(xn3[4, : ], xn3[5, :], aspect_ratio = :equal, label="Earth", lc=:green, frame=:box)
p2 = plot(xn3[1, : ], xn3[2, :], aspect_ratio = :equal, label = "Sun", lc=:red, frame = :box)
plot(p1, p2, size = [800, 400])

In [ ]:
com = ((ms .* xn3[1:3, :]) .+ (me .* xn3[4:6, :])) ./ (ms+me)
plot(com[1,:], com[2, :])#, aspect_ratio = :equal)

In [ ]:
Diagonal([1,2,3])

In [ ]:
24*365